<a href="https://colab.research.google.com/github/ArtificialCoincidence/id2223_lab2/blob/main/chinese_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare work

View GPU specification

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Install packages

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Link to huggingface

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Load data

Download data from huggingface

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="test", use_auth_token=True)

print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 7159it [00:00, 71574.71it/s]
Reading metadata...: 29056it [00:00, 109455.97it/s]


...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10581it [00:00, 72943.40it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10460it [00:00, 104593.87it/s]
Reading metadata...: 25677it [00:00, 132571.60it/s]
Reading metadata...: 39378it [00:00, 134593.84it/s]
Reading metadata...: 52838it [00:00, 130606.82it/s]
Reading metadata...: 65916it [00:00, 122687.12it/s]
Reading metadata...: 78262it [00:00, 121914.36it/s]
Reading metadata...: 90502it [00:00, 117676.79it/s]
Reading metadata...: 103299it [00:00, 120776.03it/s]
Reading metadata...: 116626it [00:00, 124532.56it/s]
Reading metadata...: 129743it [00:01, 126522.99it/s]
Reading metadata...: 143088it [00:01, 128601.43it/s]
Reading metadata...: 155979it [00:01, 128235.35it/s]
Reading metadata...: 169040it [00:01, 128944.28it/s]
Reading metadata...: 181950it [00:01, 123943.17it/s]
Reading metadata...: 195432it [00:01, 127109.58it/s]
Reading metadata...: 208557it [00:01, 127879.00it/s]
Reading metadata...: 222730it [00:01, 131962.05it/s]
Reading metadata...: 235957it [00:01, 129365.95it/s]
Reading meta

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 5860it [00:00, 58590.42it/s]
Reading metadata...: 21302it [00:00, 91584.47it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2088: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Remove some factors

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

Save common_voice to local

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/common_voice.csv"
common_voice.save_to_disk(output_dir)

Saving the dataset (0/3 shards):   0%|          | 0/39637 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10581 [00:00<?, ? examples/s]

# Prepare Feature Extractor, Tokenizer and Data

Load WhisperFeatureExtractor

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Load WhisperTokenizer

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Combine To Create A WhisperProcessor

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")

Prepare Data

In [ ]:
!pip install datasets>=2.6.1

In [ ]:
from datasets import load_from_disk, DatasetDict
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/common_voice.csv"

common_voice = load_from_disk(output_dir, keep_in_memory=True)
print(common_voice["train"][0])

Mounted at /content/drive
{'audio': {'path': 'common_voice_zh-CN_33211332.mp3', 'array': array([-6.82121026e-13, -2.27373675e-12, -2.27373675e-12, ...,
        1.21667399e-05,  3.23003678e-06, -2.43066324e-07]), 'sampling_rate': 48000}, 'sentence': '性喜温暖润湿气候且耐寒。'}


In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': 'common_voice_zh-CN_33211332.mp3', 'array': array([ 5.09317033e-11, -7.27595761e-12, -6.54836185e-11, ...,
       -5.96661994e-06,  2.71382887e-05,  1.29687978e-05]), 'sampling_rate': 16000}, 'sentence': '性喜温暖润湿气候且耐寒。'}


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
from datasets import load_dataset
from google.colab import drive
drive.mount('/content/drive')
train_cache_file = "/content/drive/MyDrive/train_cache_file"
validation_cache_file = "/content/drive/MyDrive/validation_cache_file"
test_cache_file = "/content/drive/MyDrive/test_cache_file"
other_cache_file = "/content/drive/MyDrive/other_cache_file"
invalidated_cache_file = "/content/drive/MyDrive/invalidated_cache_file"
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1,
                                cache_file_names={"train": train_cache_file, "validation": validation_cache_file, "test": test_cache_file, "other": other_cache_file, "invalidated": invalidated_cache_file})

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Save mapped common_voice to local

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/common_voice_mapped.csv"
common_voice.save_to_disk(output_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving the dataset (0/77 shards):   0%|          | 0/39637 [00:00<?, ? examples/s]

Saving the dataset (0/21 shards):   0%|          | 0/10581 [00:00<?, ? examples/s]

# Training preparation

In [ ]:
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets>=2.6.1
!pip install evaluate>=0.30
!pip install jiwer

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-hckeav9w
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-hckeav9w
  Resolved https://github.com/huggingface/transformers.git to commit 58e7f9bb2faf30622c9bead7adf472ac59f3d301
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8178366 sha256=1111152292c0d7f9b93aed8107c7c595e7f57e3f041a22f6214ccfd6e95b67e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-_ou5puy3/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Cloning https://github.com

Load WhisperTokenizer

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Combine To Create A WhisperProcessor

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Load mapped common_voice

In [ ]:
from datasets import load_from_disk, DatasetDict
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/common_voice_mapped.csv"

common_voice = load_from_disk(output_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define a Data Collator

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Initialize the collator

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Evaluation Metrics

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Load a Pre-Trained Checkpoint

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

Define the Training Configuration

In [ ]:
from transformers import Seq2SeqTrainingArguments
from google.colab import drive
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/check_points"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=1600,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_strategy="steps",
    save_steps=400, # checkpoint every 400 steps
    eval_steps=400, # evaluate every 400 steps
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    hub_strategy="checkpoint"
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
processor.save_pretrained(training_args.output_dir)

# Training

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
last_checkpoint_path = "/content/drive/MyDrive/check_points/checkpoint-1600"

if os.path.exists(last_checkpoint_path):
    trainer.train(resume_from_checkpoint=last_checkpoint_path)
    print("Resume from last checkpoint and continue training.")
else:
    trainer.train()
    print("Start training.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


Resume from last checkpoint and continue training.


In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: zh, split: test",
    "language": "zh",
    "model_name": "Whisper Small Zh - ArtificialCoincidence",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1701972070.27a835aa1e0d.2739.0:   0%|          | 0.00/5.60k [00:00<?, ?B/s]

events.out.tfevents.1701958478.31031e5a7ce9.3686.0:   0%|          | 0.00/9.64k [00:00<?, ?B/s]

'https://huggingface.co/ArtificialCoincidence/check_points/tree/main/'

# gradio

In [3]:
!pip install fastapi==0.103.2
!pip install gradio==3.48.0
!pip install requests
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
import gradio as gr
import requests
import os
import pytube as pt

youtube_file_path = "/content/drive/MyDrive/videos/youtube_video"

pipe = pipeline(model="ArtificialCoincidence/check_points")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

def _return_yt_html_embed(yt_url):
    video_id = yt_url.split("?v=")[-1]
    HTML_str = (
        f'<center> <iframe width="500" height="320" src="https://www.youtube.com/embed/{video_id}"> </iframe>'
        " </center>"
    )
    return HTML_str

def transcribe_video(yt_url):
    yt = pt.YouTube(yt_url)
    html_embed_str = _return_yt_html_embed(yt_url)
    stream = yt.streams.filter(only_audio=True)[0]
    stream.download(filename=youtube_file_path)

    text = transcribe(youtube_file_path)

    return text

iface = gr.Blocks()

microphone_trans = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath", optional=True),
    outputs="text",
    title="Whisper Chinese",
    description="Realtime demo for Chinese speech recognition using a fine-tuned Whisper small model.",
)

video_trans = gr.Interface(
    fn=transcribe_video,
    inputs=gr.Textbox(lines=1, placeholder="Paste the URL to a video here", label="video url"),
    outputs="text",
    title="Whisper Chinese",
    description="Realtime demo for Chinese speech recognition using a fine-tuned Whisper small model.",
)

with iface:
    gr.TabbedInterface([microphone_trans, video_trans], ["Transcribe Microphone", "Transcribe Video"])
iface.launch(debug=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
<ipython-input-4-138ac5d09813>:37: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.Audio(source="microphone", type="filepath", optional=True),
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1025: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://95b63e04ed77679759.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
